In [68]:
import json
import os
import numpy as np
import pickle
import tqdm

import gensim

In [2]:
os.getcwd()

'C:\\Users\\Chankoo\\Desktop\\GitHub\\BOAZ-projects\\airbnb-NLP'

In [4]:
os.chdir('./../airbnb-data')

In [5]:
with open('pos_review.json') as fp:
    review = json.load(fp)

In [8]:
len(review)

30569

In [6]:
key_lst = list(review.keys())
sampled_key_lst = key_lst[:100]
review2 = {}
for k in sampled_key_lst:
    review2[k] = review[k]

In [7]:
len(review2)

100

In [9]:
del review

In [10]:
with open('voca_freq_dic.json') as fp:
    voca_freq_dic = json.load(fp)

In [11]:
len(voca_freq_dic)

223731

In [12]:
voca_freq_dic2 = {}
for k,v in voca_freq_dic.items():
    if v>9:
        voca_freq_dic2[k] = v

In [13]:
len(voca_freq_dic2)

22384

In [14]:
del voca_freq_dic

In [124]:
ldamodel = gensim.models.ldamodel.LdaModel.load('pass_10_iter_3000.lda')

In [85]:
with open('pass_10_iter_3000.lda.id2word','rb') as fp:
    id2word = pickle.load(fp)

In [91]:
K=6

In [95]:
id2word.id2token.get(9)

'room_N'

In [103]:
aspect_wordid = []
for k in range(K):
    wid, p = zip(*ldamodel.get_topic_terms(k,topn=30))
    aspect_wordid.append(wid)

In [104]:
aspect_keywords = []
for kth_keys in aspect_wordid:
    kth_keywords = []
    for wid in kth_keys:
        kth_keywords.append(id2word.id2token.get(wid))
    aspect_keywords.append(kth_keywords)

In [111]:
len(aspect_keywords)

6

In [112]:
def get_aspect_terms(aspects, vocab_dict):
    aspect_terms = []
    w_notfound = []
    
    for aspect_kws in aspects:
        aspect = []
        for w in aspect_kws:
            if w in vocab_dict:
                aspect.append(w)
            else:
                w_notfound.append(w)
        aspect_terms.append(aspect)
    #We are only using one hotel review file, as we keep inceasing the number of files words not found will decrease.
    # print "Words not found in vocab:", ' '.join(w_notfound)
    return aspect_terms

In [113]:
# get_aspect_terms(aspect_keywords,voca_freq_dic2)

In [114]:
def chi_sq(a,b,c,d):
    c1 = a
    c2 = b - a
    c3 = c - a
    c4 = d - b - c + a
    nc =  d
    return nc * (c1*c4 - c2*c3) * (c1*c4 - c2*c3)/((c1+c3) * (c2+c4) * (c1+c2) * (c3+c4))

In [115]:
def chi_sq_mat():
    global aspect_words, aspect_sent, num_words
    asp_rank = np.zeros(aspect_words.shape)
    for i in range(len(aspect_terms)):
        for j in range(len(vocab)):
            asp_rank[i][j] = chi_sq(aspect_words[i][j], num_words[j], aspect_sent[i], len(sent))
    return asp_rank

In [116]:
aspect_terms = get_aspect_terms(aspect_keywords,voca_freq_dic2)

In [117]:
list(map(len,aspect_terms)) # aspect keywords는 5번 aspect의 1단어빼고 voca_freq_dic2에 포함됨 

[30, 30, 30, 30, 29, 30]

In [118]:
def aspect_segmentaion(review_dic,voca_freq_dic):
    #INPUT
    review_sent = [] # 문장, 단어단위로 토크나이징된 리뷰 담은 리스트
    for rev_lst in review_dic.values():
        for rev in rev_lst:
            review_sent.append(rev[1])
    
    all_ratings = []
    for rev_lst in review_dic.values():
        for rev in rev_lst:
            all_ratings.append(rev[0])

    #selection threshold
    p = 5
    
    #Iterations 
    # I = 10
    I = 1

#     #Create Vocabulary
#     #review_sent, review_actual, only_sent = parse_to_sentence(reviews)
#     #vocab, #vocab_dict = create_vocab(only_sent)

    vocab = list(voca_freq_dic.keys())
    #Assign a number corresponding to each word. Makes counting easier.
    vocab_dict = dict(zip(vocab, range(len(vocab)))) # word를 key 로 word의 인덱스를 value로  

    #Aspect Keywords
    aspect_terms = get_aspect_terms(aspect_keywords,voca_freq_dic)

#     label_text = 
    # print aspect_terms

    #ALGORITHM
    review_labels = []
    k = len(aspect_terms) # k: 토픽 개수 10
    v = len(vocab) # v: 단어 개수 18923
    
    aspect_words = np.zeros((k,v))
    aspect_sent = np.zeros(k)
    num_words = np.zeros(v)
    #-----------------------------------------------------
    for _ in tqdm.tqdm(range(I)):
        for r in review_sent:
            labels = []
            for s in r:
                count = np.zeros(len(aspect_terms)) # 한 문장에 대해 aspect keyword가 몇개씩있는지 카운트
                
                for i,a in enumerate(aspect_terms):
                    for w in s:
                        if w in vocab_dict:
                            num_words[vocab_dict[w]] += 1
                            if w in a:
                                count[i] += 1

                if max(count) > 0:
                    la = np.where(np.max(count) == count)[0].tolist() # count 중 max인 aspect들의  인덱스 뽑아 리스트로
                    
                    labels.append(la)
                    for idx in la:
                        aspect_sent[idx] += 1
                        for w in s:
                            if w in vocab_dict:
                                aspect_words[idx][vocab_dict[w]] += 1
                else:
                    labels.append([])
                    
            review_labels.append(labels)
        
#             aspect_w_rank = chi_sq_mat()
#             new_labels = []
#             for na in aspect_w_rank:
#                 x = np.argsort(na)[::-1][:p]
#                 new_labels.append(x)
#                 for k,v in vocab_dict.items():
#                     if vocab_dict[k] in x:
#                         print(k)
#             sys.exit()
            
    return review_labels,review_sent

#     ratings_sentiment = []
#     for r in review_actual:
#         sentiment = []
#         #aspect ratings based on sentiment
#         for s in r:
#             ss = sid.polarity_scores(s)
#             sentiment.append(ss['compound'])
#         ratings_sentiment.append(sentiment)

#     #Aspect Ratings Per Review
#     aspect_ratings = []
#     for i,r in enumerate(review_labels):
#         rating = np.zeros(7)
#         count = np.zeros(7)
#         rs = ratings_sentiment[i] 
#         for j,l in enumerate(r):
#             for k in range(7):
#                 if k in l:
#                     rating[k] += rs[j]
#             for k in range(7):
#                 if count[k] != 0:
#                     rating[k] /= count[k]
#         #Map from -[-1,1] to [1,5]
#         for k in range(7):
#             if rating[k] != 0:
#                 rating[k] = int(round((rating[k]+1)*5/2))
#         aspect_ratings.append(rating)
#     return aspect_ratings, all_ratings

    # n = 0
    # print review_actual[n], '\n', review_labels[n]
    # print ratings_sentiment[n], '\n', aspect_ratings[n]
    # print len(all_ratings), len(reviews), all_ratings[0]
    # sys.exit()
    # return aspect_ratings

    # print sent[5:9], labels[5:9]
    # print zip(actual_sent, labels)[:10]
    # print zip(actual_sent, sentiment)[:10]

In [46]:
omit_word = []
for k,rev_lst in review.items():
    for rev in rev_lst:
        for i,sent in enumerate(rev[1]):
            sent_tmp =[]
            for word in sent:
                if word in voca_freq_dic2:
                    sent_tmp.append(word)
                else:
                    omit_word.append(word)
            rev[1][i] = sent_tmp
            

In [119]:
%%time
review_labels2,review_sents2 = aspect_segmentaion(review2,voca_freq_dic2)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.59s/it]


Wall time: 1.62 s


In [120]:
review_labels2 = np.asarray(review_labels2)
review_sents2 = np.array(review_sents2)

In [49]:
# with open('review_labels.lst','wb') as fp:
#     pickle.dump(review_labels,fp)
# with open('review_sents.lst','wb') as fp:
#     pickle.dump(review_sent,fp)

In [121]:
review_labels2[1][0]

[0]

In [122]:
review_sents2[1][0]

['room_N',
 'spacious_J',
 'spacious_J',
 'bedroom_N',
 'see_V',
 'windowwhen_V',
 'go_V',
 'caught_J',
 'typhoon_N',
 'fortunately_R',
 'room_N',
 'occupy_V',
 'doubledecked_J',
 'windowsthe_N',
 'room_N',
 'modern_J',
 'technologicaloverall_N',
 'pretty_R',
 'good_J']

In [127]:
# LDA 의 예측과 비교

doc = review_sents2[1][0]
bow = ldamodel.id2word.doc2bow(doc)
ldamodel[bow]

[(0, 0.43307602),
 (1, 0.043798342),
 (2, 0.30630082),
 (3, 0.13603733),
 (4, 0.02652961),
 (5, 0.054257885)]

리뷰에 대해 aspect별로 word-frequency matrix를 만들어주자( *W_d_ij* )

In [128]:
review_cnt = 0
for rev_lst in review2.values():
    review_cnt += len(rev_lst)

In [129]:
review_cnt

5543

In [130]:
D = review_cnt # 리뷰수
K = len(aspect_terms) # aspect 수
N = len(voca_freq_dic2) # 단어수
D*K*N

744447072

In [55]:
# W = np.ndarray((D,K,N),dtype = 'int16')

MemoryError: 

In [131]:
W = {} # K로 나눠 딕셔너리 만들자
for i in range(K):
    W['w_'+str(i)] = np.ndarray((D,N),dtype = 'int16')

In [134]:
W

{'w_0': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int16),
 'w_1': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int16),
 'w_2': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int16),
 'w_3': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int16),
 'w_4': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0,

In [132]:
def kth_count(sent_labels, sents,k,freq_dic_kth):
    for idx,label in enumerate(sent_labels):
        if k in label: # kth aspect이면
            for word in sents[idx]:
                if word in freq_dic_kth:
                    freq_dic_kth[word] += 1
                else:
                    continue
        

def make_kth_mat(k):
    global W,D # word freq 담을 전역변수 W. kth aspect 의 freq은 w_k (np.array)에 담는다/ 리뷰개수 D
    for d in tqdm.tqdm(range(D)):# 리뷰개수만큼 돌면서
        freq_dic_kth = dict.fromkeys(voca_freq_dic2.keys(),0) # kth freq dic 초기화
        kth_count(review_labels2[d],review_sents2[d],k,freq_dic_kth) # freq_dic_kth 을 완성
        words,freqs = zip(*sorted(freq_dic_kth.items(),key = lambda kv:kv[0])) # freq_dic_kth를 사전순으로 정렬해 단어와 freq를 각각 뽑는다
        W['w_'+str(k)][d] = np.array(freqs,dtype='int16')


In [136]:
for k in range(K):
    make_kth_mat(k)

100%|██████████████████████████████████████████████████████████████████████████████| 5543/5543 [04:10<00:00, 22.13it/s]


In [140]:
# with open('W_6_100.mat','wb') as fp:
#     pickle.dump(W,fp)

In [51]:
with open('W_6_100.mat','rb') as fp:
    W = pickle.load(fp)

In [142]:
for i in range(K): # k aspect별로 단어 freq의 합을 찍어보자
    print('{}th aspect'.format(i))
    print(aspect_keywords[i][:5])
    print('sum of freq of words __',np.sum(W['w_'+str(i)]),'__')

0th aspect
['room_N', 'house_N', 'nice_J', 'clean_J', 'really_R']
sum of freq of words __ 38579 __
1th aspect
['station_N', 'convenient_N', 'restaurant_N', 'easy_J', 'area_N']
sum of freq of words __ 22083 __
2th aspect
['place_N', 'great_J', 'good_J', 'location_N', 'stay_N']
sum of freq of words __ 34118 __
3th aspect
['well_R', 'everything_N', 'need_V', 'small_J', 'people_N']
sum of freq of words __ 15631 __
4th aspect
['take_V', 'check_N', 'check_V', 'wifi_N', 'arrive_V']
sum of freq of words __ 7347 __
5th aspect
['host_N', 'get_V', 'kind_N', 'lot_N', 'provide_V']
sum of freq of words __ 17506 __
